In [5]:
import os   
from dataclasses import dataclass
from pathlib import Path
from src.mlproject.constant import *
from src.mlproject.utils.common import read_yaml, create_directories, get_size
import urllib.request as request #To download the data from the URL
import zipfile #To unzip the downloaded data file
from src.mlproject import logger

In [6]:
%pwd

'c:\\Users\\Parag Jadhav\\Downloads\\ML_Deployment\\red_wine_qualit_prediction\\research'

In [7]:
os.chdir("../")

In [8]:
%pwd

'c:\\Users\\Parag Jadhav\\Downloads\\ML_Deployment\\red_wine_qualit_prediction'

In [9]:
@dataclass(frozen=True)
class DataIngestionConfig:
    '''
    This is en entity. It is a custom return type of the function using "dataclass". 
    Similar to the "confibox". This is inbuilt return type of the function.
    Below variables from config.yaml with its data type.
    '''
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH, #constant/__init__.py file gives the path of config.yaml file
        params_filepath = PARAMS_FILE_PATH, 
        schema_filepath = SCHEMA_FILE_PATH): 

        self.config = read_yaml(config_filepath) #whatever is present inside the config.yaml file, it will read the it by providing the path of the config.yaml file.
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root]) #create the "artifacts" directory and name is present inside the config.yaml file.


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir]) #create the "data_ingestion" directory inside the "artifacts" directory and name is present inside the config.yaml file.

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [12]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-05-10 21:37:35,573: line_number-43: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-10 21:37:35,578: line_number-43: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-10 21:37:35,584: line_number-43: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-05-10 21:37:35,587: line_number-63: INFO: common: created directory at: artifacts]
[2024-05-10 21:37:35,590: line_number-63: INFO: common: created directory at: artifacts/data_ingestion]
[2024-05-10 21:37:36,820: line_number-12: INFO: 3878650913: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 25123
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "84a92c470bf03cc88c9f3eefef7d75b6b306388755483c800dd0bad1204da074"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=